In [1]:
%store -r df_listings

In [2]:
df_listings.shape

(13945, 79)

In [3]:
%store -r df_calendar_cleaned

In [4]:
%store -r merged_neighborhoods

no stored variable or alias merged_neighborhoods


In [5]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
df_listings['price'] = df_listings['price'].astype(str).str.replace('[\$,]', '', regex=True)

In [8]:
import pandas as pd
df_listings['price'] = pd.to_numeric(df_listings['price'], errors='coerce')

In [9]:
df_listings.dropna(subset=['price'],inplace=True)

In [10]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8898 entries, 0 to 13944
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            8898 non-null   int64  
 1   listing_url                                   8898 non-null   object 
 2   scrape_id                                     8898 non-null   int64  
 3   last_scraped                                  8898 non-null   object 
 4   source                                        8898 non-null   object 
 5   name                                          8898 non-null   object 
 6   description                                   8518 non-null   object 
 7   neighborhood_overview                         4238 non-null   object 
 8   picture_url                                   8898 non-null   object 
 9   host_id                                       8898 non-null   int64

In [11]:
listings = df_listings.copy()

In [12]:
bool_cols = [
    "host_is_superhost", "instant_bookable", "has_availability",
    "host_has_profile_pic", "host_identity_verified"
]
for col in bool_cols:
    listings[col] = (
        listings[col]
        .map({"t": True, "f": False})          
        .fillna(False)                      
        .astype("bool")
    )

In [13]:
date_cols = ["host_since", "first_review", "last_review"]
listings[date_cols] = listings[date_cols].apply(
    pd.to_datetime, errors="coerce"
)

In [14]:
num_cols = [
    "accommodates", "bedrooms", "beds", "bathrooms",
    "minimum_nights", "maximum_nights",
    "availability_30","availability_60","availability_90", "availability_365",
    "number_of_reviews", "review_scores_rating"
]
listings[num_cols] = (
    listings[num_cols]
      .apply(pd.to_numeric, errors="coerce")
)

In [15]:
listings['price'].describe()

count     8898.000000
mean       128.763880
std        316.080704
min          9.000000
25%         65.000000
50%         94.000000
75%        138.000000
max      10006.000000
Name: price, dtype: float64

In [16]:
for c in ["instant_bookable", "has_availability",
          "host_has_profile_pic", "host_identity_verified"]:
    listings[c] = listings[c].map({'t': True, 'f': False}).fillna(False)

# host_acceptance_rate  "%→float0-1"  +  median fill
lst = listings
lst['host_acceptance_rate'] = (
    lst['host_acceptance_rate'].str.rstrip('%').astype(float).div(100)
)
lst['host_acceptance_rate'].fillna(lst['host_acceptance_rate'].median(), inplace=True)


In [17]:
lst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8898 entries, 0 to 13944
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            8898 non-null   int64         
 1   listing_url                                   8898 non-null   object        
 2   scrape_id                                     8898 non-null   int64         
 3   last_scraped                                  8898 non-null   object        
 4   source                                        8898 non-null   object        
 5   name                                          8898 non-null   object        
 6   description                                   8518 non-null   object        
 7   neighborhood_overview                         4238 non-null   object        
 8   picture_url                                   8898 non-null   object    

In [18]:
%store lst

Stored 'lst' (DataFrame)


In [19]:
#host_since, first_review, andb last_review for feature engineering

In [20]:
#host_since, first_review, and last_review for feature engineering 